In [359]:
import cv2
import numpy as np
import pandas as pd
import random
#from matplotlib import pyplot as plt

pairs_count = 100
mask_file = 'IMG_1119_mask.jpg'
image_file = 'IMG_1119.JPG'
folder_to_save = 'images/'

In [360]:
# generate Random Point on a picture - define left top angle of cropped image
# define range of possible values for the side of the square
def generate_random_point(page_extr, detail_extr, image):

    # random point should be to the left from the detail and higher than the detail
    rp_col = random.randint(page_extr.loc['left']['col'], detail_extr.loc['left']['col'])
    rp_row = random.randint(page_extr.loc['top']['row'], detail_extr.loc['top']['row'])
    min_side = max(detail_extr.loc['right']['col'] - rp_col, detail_extr.loc['bot']['row'] - rp_row)
    max_side = min(page_extr.loc['right']['col'] - rp_col, page_extr.loc['bot']['row'] - rp_row)
    
    # it must be possible to construct a square inside the image, starting from random point
    # random point must be inside the image itself
    # a square must iclude the whole detail
    while (max_side <= min_side) \
    or (image[rp_row, rp_col] <> 0) \
    or (image[rp_row, rp_col + min_side] <> 0) \
    or (image[rp_row + min_side, rp_col] <> 0) \
    or (image[rp_row + min_side, rp_col + min_side] <> 0):

        rp_col = random.randint(page_extr.loc['left']['col'], detail_extr.loc['left']['col'])
        rp_row = random.randint(page_extr.loc['top']['row'], detail_extr.loc['top']['row'])

        min_side = max(detail_extr.loc['right']['col'] - rp_col, detail_extr.loc['bot']['row'] - rp_row)
        max_side = min(page_extr.loc['right']['col'] - rp_col, page_extr.loc['bot']['row'] - rp_row)

    return rp_col, rp_row, min_side, max_side

In [361]:
# generate random side of the square to crop
def generate_random_side(image, rp_row, rp_col, min_side, max_side):
    r_side = random.randint(min_side, max_side)
    
    # searching for the random size of square side, based on the condition:
    # all of the angles of cropped square must be inside the image
    while (image[rp_row, rp_col+r_side] <> 0) \
    or (image[rp_row+r_side, rp_col+r_side] <> 0) \
    or (image[rp_row+r_side, rp_col] <> 0):
        r_side = random.randint(min_side, max_side)

    return r_side

In [362]:
def empty_coordinates_dataframe():
    columns = ['row', 'col']
    index = ['left', 'right', 'top', 'bot']
    return pd.DataFrame(index=index, columns=columns)

In [363]:
# count extreme values of the area of the specified color
def get_extremes(image, color):
    pixels_array = np.argwhere(image == color)
    extr_arr = empty_coordinates_dataframe()
    extr_arr.set_value('left','col',pixels_array[:,1].min())
    extr_arr.set_value('right','col',pixels_array[:,1].max())
    extr_arr.set_value('top','row',pixels_array[:,0].min())
    extr_arr.set_value('bot','row',pixels_array[:,0].max())
    return extr_arr

In [364]:
def search_crop_parameters(image):
    
    # find extremes for the page and for the detail
    # the page area is identified by black pixels
    page_extr = get_extremes(image, 0)
    # the detail area is identified by white pixels
    detail_extr = get_extremes(image, 255)

    rp_col, rp_row, min_side, max_side = generate_random_point(page_extr, detail_extr, image)
    r_side = generate_random_side(image, rp_row, rp_col, min_side, max_side)
 
    return rp_row, rp_col, r_side

In [365]:
def crop_image(image, point_row, point_col, side):
    cut_image = image[point_row : point_row + side , point_col : point_col + side]
    resized_image = cv2.resize(cut_image, (256, 256)) 
    return resized_image

In [366]:
# rotate image without cutting angles
def rotate_image(image, angle):
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY
    # mark an area outside of a page with gray
    return cv2.warpAffine(image, M, (nW, nH), borderValue=100)

In [367]:
mask = cv2.imread(mask_file, 0)
img = cv2.imread(image_file, -1)

for i in range(pairs_count):

    # choose randon angle and rotate the image
    rand_angle = random.uniform(0, 360)
    rotated_mask = rotate_image(mask, rand_angle)
    rotated_image = rotate_image(img, rand_angle)
    
    # we can also flip the image
    flip_action = random.choice([0,1,-1,-1])
    if flip_action >= 0:
        rotated_mask = cv2.flip(rotated_mask, flip_action)
        rotated_image = cv2.flip(rotated_image, flip_action)
    
    # define parameters of Starting Point and size of square side to crop
    sp_row, sp_col, side = search_crop_parameters(rotated_mask)
    
    # crop and save mask and image
    cut_mask = crop_image(rotated_mask, sp_row, sp_col, side)
    cut_image = crop_image(rotated_image, sp_row, sp_col, side)
    cv2.imwrite(folder_to_save + str(i).zfill(3) + '_mask.jpg', cut_mask)
    cv2.imwrite(folder_to_save + str(i).zfill(3) + '_img.jpg', cut_image)

    #imgplot = plt.imshow(cut_image, 'gray')
    #plt.show()